***
## Кастомный шаблон для PasswordChangeView

CBV **PasswordChangeView** возвращает страницу с формой, через которую залогиненный пользователь может поменять пароль. 

За отображение этой страницы отвечает шаблон *registration/password_change_form.html*; разместите в нём такой код:

```html
{% extends "base.html" %}
{% load django_bootstrap5 %}

{% block content %}
  <div class="card col-4 m-3">
    <div class="card-header">
      Поменять пароль
    </div>
    <div class="card-body">
      <form method="post">
        {% csrf_token %}
        {% bootstrap_form form %}
        {% bootstrap_button button_type="submit" content="Поменять пароль" %}
      </form>
    </div>
  </div>
{% endblock %} 
```

Адрес страницы для смены пароля — */auth/password_change/*; эта страница доступна только аутентифицированным пользователям. Если на эту страницу попытается войти незалогиненный пользователь — Django перенаправит его на страницу логина. 

Однако по умолчанию в Django страница логина ожидается по адресу */accounts/profile/*, а не */auth/profile/*, как в настройках нашего проекта. 

Адрес страницы логина можно указать в настройках проекта, в константе `LOGIN_URL`. В ней указывается `name` страницы логина. Добавьте в settings.py константу `LOGIN_URL`:

```py
# acme_project/settings.py
...

LOGIN_URL = 'login' 
```

Ещё раз зайдите без логина на страницу http://127.0.0.1:8000/auth/password_change/: вас переадресует уже на правильный адрес: http://127.0.0.1:8000/auth/login/?next=/auth/password_change/. 

А после отправки формы вас перенаправит на страницу смены пароля:

![alt text](https://pictures.s3.yandex.net/resources/207_1682601725.png)

Подсказки, которые выведены в форме, генерируются вместе с формой, дополнительный текст в шаблон добавлять не нужно.

***
## Кастомный шаблон для PasswordChangeDoneView

Этот CBV возвращает страницу с информацией об успешной смене пароля: именно на эту страницу переадресуется пользователь, заполнивший форму на странице */auth/password_change/*. 

Класс **PasswordChangeDoneView** использует шаблон *registration/password_change_done.html*. Перенесите в этот шаблон код из листинга:

```py
{% extends "base.html" %}

{% block content %}
  <h2>Пароль успешно изменён!</h2>
  <p><a href="{% url 'pages:homepage' %}">На главную</a></p>
{% endblock %} 
```

Эта страница доступна по адресу http://127.0.0.1:8000/auth/password_change/done/.

***
## Восстановление пароля

Если пользователь забыл пароль — он обращается к странице */auth/password_reset/* — именно на неё ведёт ссылка «Забыли пароль?» на странице логина.

Там он должен ввести свой email, на который придёт письмо со ссылкой для восстановления пароля. В проекте ACME у обычного пользователя не будет возможности установить себе email самостоятельно — он сможет это сделать только через админа системы. В финальном проекте вы создадите страницу, где пользователь сам сможет указать свой email. А пока что для простоты опустим этот шаг.

Процедура восстановления пароля выполняется в несколько шагов: в ней задействованы четыре страницы и четыре CBV:

![alt text](https://pictures.s3.yandex.net/resources/S04_06_1685550521.png)


***
## Кастомный шаблон для PasswordResetView

CBV **PasswordResetView** возвращает страницу с формой, куда пользователь должен ввести свой почтовый адрес, с которым он зарегистрирован на сайте. 

В шаблон *registration/password_reset_form.html* вставьте код:

```html
{% extends "base.html" %}
{% load django_bootstrap5 %}

{% block content %}
  <div class="card col-4 m-3">
    <div class="card-header">
      Восстановить пароль
    </div>
    <div class="card-body">
      <form method="post">
        {% csrf_token %}
        {% bootstrap_form form %}
        {% bootstrap_button button_type="submit" content="Отправить" %}
      </form>
    </div>
  </div>
{% endblock %} 
```

Проверить вид страницы можно по адресу http://127.0.0.1:8000/auth/password_reset/.

![alt text](https://pictures.s3.yandex.net/resources/208_1682601764.png)

Перейдите на эту страницу, введите email своего суперпользователя и отправьте форму. 

Проверьте директорию *sent_emails*: в ней должен появиться текстовый файл с письмом. В письме будет уникальная ссылка на сброс пароля:

```
http://127.0.0.1:8000/auth/reset/Mg/bhrlrp-5f55963b8c348443f6837450115bc3d3/ 
```

В ссылке зашифрован id пользователя и токен, который практически нереально подобрать путём перебора. Это позволяет быть уверенным в том, что пароль меняет именно получатель письма, а не злоумышленник.

Если в форму будет введён email, которого нет в базе данных пользователей, то Django ничего не сообщит об этом; по поведению интерфейса невозможно отличить, правильный ли адрес был введён в форму восстановления пароля. 

Таким образом Django защищает адреса пользователей: с помощью сервиса восстановления пароля невозможно узнать, есть ли в базе тот или иной email. 

***
## Кастомный шаблон для PasswordResetDoneView

После отправки формы пользователь переадресуется на страницу с сообщением «вам отправлено письмо со ссылкой». За отображение этой страницы отвечает CBV **PasswordResetDoneView**; страница доступна по адресу */auth/password_reset/done/*.

Стандартный шаблон этого CBV называется *registration/password_reset_done.html*; код этого шаблона должен быть таким:

```html
{% extends "base.html" %}

{% block content %}
  <h2>Письмо с инструкциями по восстановлению пароля отправлено</h2>
  <p>
    Мы отправили вам инструкцию по установке нового пароля
    на указанный адрес электронной почты (если в нашей базе данных есть такой адрес). 
    Вы должны получить её в ближайшее время.
  </p>
  <p>
    Если вы не получили письмо, пожалуйста,
    убедитесь, что вы ввели адрес, с которым Вы зарегистрировались,
    и проверьте папку со спамом.
  </p>
  <p><a href="{% url 'pages:homepage' %}">На главную</a></p>
{% endblock %} 
```

Проверить вид страницы можно по адресу http://127.0.0.1:8000/auth/password_reset/done/.

![alt text](https://pictures.s3.yandex.net/resources/209_1682601790.png)


***
## Кастомный шаблон для PasswordResetConfirmView

Пользователь получил письмо со ссылкой и перешёл по ней. Ссылка в письме соответствует маршруту 

```py
# django/contrib/auth/urls.py
...
path(
    'reset/<uidb64>/<token>/', 
    views.PasswordResetConfirmView.as_view(), 
    name='password_reset_confirm'
), 
```

Сочетание переменных `<uidb64>/<token>/` в ссылке для восстановления пароля — это одноразовый уникальный код. Django проверяет полученные в запросе id и токен, и если они валидны, то CBV **PasswordResetConfirmView** возвращает пользователю форму для изменения пароля. 

По умолчанию CBV PasswordResetConfirmView вызывает шаблон *registration/password_reset_confirm.html*. 

Вставьте в этот шаблон код из листинга:

```html
{% extends "base.html" %}
{% load django_bootstrap5 %}

{% block content %}
  <div class="card col-4 m-3">
    <div class="card-header">
      Восстановление пароля
    </div>
    <div class="card-body">
      <form method="post">
        {% csrf_token %}
        {% bootstrap_form form %}
        {% bootstrap_button button_type="submit" content="Поменять пароль" %}
      </form>
    </div>
  </div>
{% endblock %} 
```

Проверьте работу формы: перейдите по ссылке из письма. Вас переадресует на страницу с адресом */auth/reset/<uidb64>/set-password/* ; на этой странице будет форма для смены пароля.

![alt text](https://pictures.s3.yandex.net/resources/210_1682601810.png)


***
## Кастомный шаблон для PasswordResetCompleteView

Последний этап восстановления пароля — уведомление об успешном завершении процесса. Это просто страница с текстом, за её отображение отвечает CBV **PasswordResetCompleteView**. Этот класс работает с шаблоном *registration/password_reset_complete.html*. 

Добавьте в шаблон код из листинга:

```html
{% extends "base.html" %}

{% block content %}
  <h2>Восстановление пароля завершено</h2>
  <p>Ваш новый пароль сохранён. Теперь вы можете войти.</p>
  <p><a href="{% url 'login' %}">Войти</a></p>
{% endblock %} 
```

Увидеть эту страницу можно по адресу http://127.0.0.1:8000/auth/reset/done/.

В замене шаблонов нет ничего сверхъестественного, достаточно найти в документации название нужного шаблона по умолчанию и создать шаблон с аналогичным именем в директории шаблонов registration. 